In [77]:
%config Completer.use_jedi = False

import pandas as pd
import json
import os
import datetime
from time import mktime
import plotly.express as px
import plotly.graph_objects as go
import utm

In [78]:
data_path = "../data/J2-1393/"

In [79]:
with open("../data/fields.json") as fields_file:
    fields = json.load(fields_file)

In [80]:
fields['CT2']

['Label',
 'Date',
 'Time',
 'Label',
 'Temperature',
 'Conductivity',
 'Pressure',
 'Salinity',
 'Sound Velocity']

In [81]:
ct2_fields = ["lump"] + fields["CT2"][5:]

In [82]:
ct2_0 = pd.read_csv(os.path.join(data_path, "ct2/20211122_0000.CT2"), header = None, names=ct2_fields)


ct2_0.lump = ct2_0["lump"].apply(lambda x: x.split())

ct2_0 = pd.concat([pd.DataFrame(ct2_0.lump.tolist(), columns=fields["CT2"][:5]), ct2_0], axis=1)
ct2_0 = ct2_0.drop("lump", axis=1)

ct2_0

,Label,Date,Time,Label,Temperature,Conductivity,Pressure,Salinity,Sound Velocity
0,CT2,2021/11/22,00:00:00.758,SB19,2.9385,3.20353,1756.615,34.5921,1490.662
1,CT2,2021/11/22,00:00:01.758,SB19,2.9385,3.20353,1756.605,34.5920,1490.662
2,CT2,2021/11/22,00:00:02.759,SB19,2.9385,3.20352,1756.600,34.5920,1490.661
3,CT2,2021/11/22,00:00:03.760,SB19,2.9385,3.20352,1756.616,34.5920,1490.662
4,CT2,2021/11/22,00:00:04.758,SB19,2.9385,3.20354,1756.620,34.5922,1490.662
...,...,...,...,...,...,...,...,...,...
3594,CT2,2021/11/22,00:59:55.826,SB19,2.9333,3.20249,1741.727,34.5923,1490.391
3595,CT2,2021/11/22,00:59:56.825,SB19,2.9333,3.20249,1741.724,34.5923,1490.390
3596,CT2,2021/11/22,00:59:57.825,SB19,2.9333,3.20248,1741.744,34.5922,1490.391
3597,CT2,2021/11/22,00:59:58.826,SB19,2.9332,3.20250,1741.748,34.5926,1490.391


In [116]:
csv_path = os.path.join(data_path, "csv/20211122_0000.CSV")
with open(csv_path) as csv_file:
    csv_header = csv_file.readline()[2:]
    
csv_header = csv_header.strip().split(',')

# There are two depth columns,
# So we have to make them both unique
csv_header[-6] += '_2'
csv_header += ["none"]

csv_0 = pd.read_csv(csv_path, skiprows=2, header=None, index_col=None)
csv_0.columns = csv_header
csv_0.index = csv_0["time(seconds since Jan 1 1970)"]

csv_0[['northing', 'easting']] = csv_0.apply(
    lambda row: utm.from_latlon(row['latitude(deg)'], row['longitude(deg)'])[:2], 
    axis=1
).tolist()

In [117]:
def date_time_to_unix(date_str, time):
    reformatted_date = '-'.join(date_str.split('/'))
    d = datetime.date.fromisoformat(reformatted_date)
    # Converts to unix UTC time
    time_for_day = (d.toordinal() - datetime.date(1970, 1, 1).toordinal()) * 24*60*60
    

    time_piece = time.split(":")
    time_of_day = datetime.timedelta(
        hours = int(time_piece[0]),
        minutes = int(time_piece[1]),
        seconds = float(time_piece[2])
    )
    
    return time_for_day + time_of_day.total_seconds()

In [118]:
date_time_to_unix("2021/11/22", "00:01:10.758")

1637539270.758

In [119]:
ct2_0.index = ct2_0.apply(lambda row: date_time_to_unix(row.Date, row.Time), axis=1)

In [120]:
ct2_0

,Label,Date,Time,Label,Temperature,Conductivity,Pressure,Salinity,Sound Velocity
1.637539e+09,CT2,2021/11/22,00:00:00.758,SB19,2.9385,3.20353,1756.615,34.5921,1490.662
1.637539e+09,CT2,2021/11/22,00:00:01.758,SB19,2.9385,3.20353,1756.605,34.5920,1490.662
1.637539e+09,CT2,2021/11/22,00:00:02.759,SB19,2.9385,3.20352,1756.600,34.5920,1490.661
1.637539e+09,CT2,2021/11/22,00:00:03.760,SB19,2.9385,3.20352,1756.616,34.5920,1490.662
1.637539e+09,CT2,2021/11/22,00:00:04.758,SB19,2.9385,3.20354,1756.620,34.5922,1490.662
...,...,...,...,...,...,...,...,...,...
1.637543e+09,CT2,2021/11/22,00:59:55.826,SB19,2.9333,3.20249,1741.727,34.5923,1490.391
1.637543e+09,CT2,2021/11/22,00:59:56.825,SB19,2.9333,3.20249,1741.724,34.5923,1490.390
1.637543e+09,CT2,2021/11/22,00:59:57.825,SB19,2.9333,3.20248,1741.744,34.5922,1490.391
1.637543e+09,CT2,2021/11/22,00:59:58.826,SB19,2.9332,3.20250,1741.748,34.5926,1490.391


In [121]:
combined = pd.merge_asof(csv_0, ct2_0, left_index=True, right_index=True)

In [122]:
combined

,Vehicle,year,month,day,hour,minutes,seconds,time(seconds since Jan 1 1970),latitude(deg),longitude(deg),...,easting,Label,Date,Time,Label,Temperature,Conductivity,Pressure,Salinity,Sound Velocity
time(seconds since Jan 1 1970),,,,,,,,,,,,,,,,,,,,,
1.637539e+09,JAS2,2021,11,22,0,0,0.855,1.637539e+09,27.501088,-111.683608,...,3.042123e+06,CT2,2021/11/22,00:00:00.758,SB19,2.9385,3.20353,1756.615,34.5921,1490.662
1.637539e+09,JAS2,2021,11,22,0,0,1.855,1.637539e+09,27.501088,-111.683608,...,3.042123e+06,CT2,2021/11/22,00:00:01.758,SB19,2.9385,3.20353,1756.605,34.5920,1490.662
1.637539e+09,JAS2,2021,11,22,0,0,2.855,1.637539e+09,27.501088,-111.683608,...,3.042123e+06,CT2,2021/11/22,00:00:02.759,SB19,2.9385,3.20352,1756.600,34.5920,1490.661
1.637539e+09,JAS2,2021,11,22,0,0,3.855,1.637539e+09,27.501088,-111.683608,...,3.042123e+06,CT2,2021/11/22,00:00:03.760,SB19,2.9385,3.20352,1756.616,34.5920,1490.662
1.637539e+09,JAS2,2021,11,22,0,0,4.855,1.637539e+09,27.501088,-111.683608,...,3.042123e+06,CT2,2021/11/22,00:00:04.758,SB19,2.9385,3.20354,1756.620,34.5922,1490.662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.637543e+09,JAS2,2021,11,22,0,59,55.090,1.637543e+09,27.501226,-111.683487,...,3.042139e+06,CT2,2021/11/22,00:59:54.825,SB19,2.9331,3.20249,1741.732,34.5925,1490.390
1.637543e+09,JAS2,2021,11,22,0,59,56.090,1.637543e+09,27.501226,-111.683487,...,3.042139e+06,CT2,2021/11/22,00:59:55.826,SB19,2.9333,3.20249,1741.727,34.5923,1490.391
1.637543e+09,JAS2,2021,11,22,0,59,57.090,1.637543e+09,27.501226,-111.683487,...,3.042139e+06,CT2,2021/11/22,00:59:56.825,SB19,2.9333,3.20249,1741.724,34.5923,1490.390


In [90]:
combined["Temperature"] = combined["Temperature"].astype(float)

In [91]:
combined.columns

Index(['Vehicle', 'year', 'month', 'day', 'hour', 'minutes', 'seconds',
       'time(seconds since Jan 1 1970)', 'latitude(deg)', 'longitude(deg)',
       'X local(m)', 'Y local(m)', 'latitude origin(deg)',
       'longitude origin(deg)', 'X UTM(m)', 'Y UTM(m)', 'UTMZone', 'depth(m)',
       'altitude(m)', 'octans heading(deg)', 'octans pitch(deg)',
       'octans roll(deg)', 'crossbow heading(deg)', 'crossbow pitch(deg)',
       'crossbow roll(deg)', 'maggie x', 'maggie y', 'maggie z',
       'maggie total', 'conductivity', 'temperature(deg C)', 'depth(m)_2',
       'salinity', 'sound velocity', 'pressure', 'lss gain', 'lss backscatter',
       'none', 'Label', 'Date', 'Time', 'Label', 'Temperature', 'Conductivity',
       'Pressure', 'Salinity', 'Sound Velocity'],
      dtype='object')

In [92]:
fig = px.scatter_mapbox(combined, lat="latitude(deg)", lon="longitude(deg)",
                        color="Temperature", zoom=19)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":2,"b":0})
fig.show()

In [123]:
def plot_mesh():
    ring_depth = pd.read_csv("../data/ring_depth.csv", header=None, names = ['lon', 'lat', 'depth'], sep='\t')
    usable_ring_depth = ring_depth[pd.notna(ring_depth.depth)]
    usable_ring_depth = usable_ring_depth.reset_index(drop=True)
    STEP = 1000
    subsample_ring_depth = usable_ring_depth.iloc[list(range(0, len(usable_ring_depth), STEP))].reset_index(drop=True)


    subsample_ring_depth[['northing', 'easting']] = subsample_ring_depth.apply(lambda row: utm.from_latlon(row['lat'], row['lon'])[:2], axis=1).tolist()
    
    ring_mesh = go.Mesh3d(
        x=subsample_ring_depth.northing, 
        y=subsample_ring_depth.easting, 
        z=subsample_ring_depth.depth, 
        opacity=.5
    )

    return go.Figure(data=[ring_mesh])



In [124]:
fig = plot_mesh()
fig.show()

In [125]:
fig.add_scatter3d(
    x = combined['northing'].tolist(), 
    y = combined['easting'].tolist(), 
    z = combined['depth(m)'].tolist())

In [100]:
combined['northing']

KeyError: 'northing'

In [182]:
csv

,Vehicle,year,month,day,hour,minutes,seconds,time(seconds since Jan 1 1970),latitude(deg),longitude(deg),...,maggie total,conductivity,temperature(deg C),depth(m)_2,salinity,sound velocity,pressure,lss gain,lss backscatter,None
0,JAS2,2021,11,21,22,0,0.378,1.637532e+09,27.500114,-111.683222,...,43852.650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JAS2,2021,11,21,22,0,1.378,1.637532e+09,27.500114,-111.683222,...,43853.747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,JAS2,2021,11,21,22,0,2.378,1.637532e+09,27.500114,-111.683222,...,43868.988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JAS2,2021,11,21,22,0,3.378,1.637532e+09,27.500114,-111.683222,...,43888.601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,JAS2,2021,11,21,22,0,4.378,1.637532e+09,27.500114,-111.683222,...,43892.815,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,JAS2,2021,11,21,22,59,55.615,1.637536e+09,27.500955,-111.683727,...,42113.848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3596,JAS2,2021,11,21,22,59,56.615,1.637536e+09,27.500955,-111.683727,...,42142.242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3597,JAS2,2021,11,21,22,59,57.615,1.637536e+09,27.500955,-111.683727,...,42127.421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3598,JAS2,2021,11,21,22,59,58.615,1.637536e+09,27.500955,-111.683727,...,42131.196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
DAY = '20211121_2200'

In [ ]:
formats = ['csv', 'ct2', 'oos', 'obs']

In [179]:
files = os.listdir("../data/J2-1393/csv")
files.sort()
files = [x.split('.')[0] for x in files]

In [181]:
import utils
from importlib import reload

csv = utils.load_from_file(f'../data/J2-1393/csv/{DAY}.CSV')
ct2 = utils.load_from_file(f'../data/J2-1393/ct2/{DAY}.CT2')
oos = utils.load_from_file(f'../data/J2-1393/oos/{DAY}.OOS')
obs = utils.load_from_file(f'../data/J2-1393/obs/{DAY}.OBS')

ct2['unix_time'] = ct2.apply(lambda row: utils.date_time_to_unix(row['date'], row['time']), axis=1)
oos['unix_time'] = oos.apply(lambda row: utils.date_time_to_unix(row['date'], row['time']), axis=1)
obs['unix_time'] = obs.apply(lambda row: utils.date_time_to_unix(row['date'], row['time']), axis=1)

In [169]:
from scipy.interpolate import interp1d
import utils
from importlib import reload

csv = utils.load_from_file(f'../data/J2-1393/csv/{DAY}.CSV')
ct2 = utils.load_from_file(f'../data/J2-1393/ct2/{DAY}.CT2')
oos = utils.load_from_file(f'../data/J2-1393/oos/{DAY}.OOS')
obs = utils.load_from_file(f'../data/J2-1393/obs/{DAY}.OBS')

ct2['unix_time'] = ct2.apply(lambda row: utils.date_time_to_unix(row['date'], row['time']), axis=1)
oos['unix_time'] = oos.apply(lambda row: utils.date_time_to_unix(row['date'], row['time']), axis=1)
obs['unix_time'] = obs.apply(lambda row: utils.date_time_to_unix(row['date'], row['time']), axis=1)

interp_obs = interp1d(obs.unix_time, obs.obs, fill_value='extrapolate')
interp_oos = interp1d(oos.unix_time, oos.oxygen, fill_value='extrapolate')

csv['obs_proj'] = interp_obs(csv['time(seconds since Jan 1 1970)'])
csv['oxygen_proj'] = interp_oos(csv['time(seconds since Jan 1 1970)'])

csv[['northing', 'easting']] = csv.apply(
    lambda row: utm.from_latlon(row["latitude(deg)"], row["longitude(deg)"])[:2], 
    axis=1
).tolist()

FileNotFoundError: [Errno 2] No such file or directory: '../data/J2-1393/ct2/20211121_2012.CT2'

In [165]:
csv[['northing', 'easting', 'latitude(deg)', 'longitude(deg)', 'depth(m)']]

,northing,easting,latitude(deg),longitude(deg),depth(m)
0,432515.553144,3.042015e+06,27.500114,-111.683222,1730.643
1,432515.553144,3.042015e+06,27.500114,-111.683222,1730.615
2,432515.553144,3.042015e+06,27.500114,-111.683222,1730.595
3,432515.553144,3.042015e+06,27.500114,-111.683222,1730.569
4,432515.553144,3.042015e+06,27.500114,-111.683222,1730.545
...,...,...,...,...,...
3595,432466.134565,3.042109e+06,27.500955,-111.683727,1735.068
3596,432466.134565,3.042109e+06,27.500955,-111.683727,1735.069
3597,432466.134565,3.042109e+06,27.500955,-111.683727,1735.069
3598,432466.134565,3.042109e+06,27.500955,-111.683727,1735.068


In [166]:
csv

,Vehicle,year,month,day,hour,minutes,seconds,time(seconds since Jan 1 1970),latitude(deg),longitude(deg),...,salinity,sound velocity,pressure,lss gain,lss backscatter,None,obs_proj,oxygen_proj,northing,easting
0,JAS2,2021,11,21,22,0,0.378,1.637532e+09,27.500114,-111.683222,...,NaN,NaN,NaN,NaN,NaN,NaN,0.037500,NaN,432515.553144,3.042015e+06
1,JAS2,2021,11,21,22,0,1.378,1.637532e+09,27.500114,-111.683222,...,NaN,NaN,NaN,NaN,NaN,NaN,0.036120,NaN,432515.553144,3.042015e+06
2,JAS2,2021,11,21,22,0,2.378,1.637532e+09,27.500114,-111.683222,...,NaN,NaN,NaN,NaN,NaN,NaN,0.038727,NaN,432515.553144,3.042015e+06
3,JAS2,2021,11,21,22,0,3.378,1.637532e+09,27.500114,-111.683222,...,NaN,NaN,NaN,NaN,NaN,NaN,0.033750,NaN,432515.553144,3.042015e+06
4,JAS2,2021,11,21,22,0,4.378,1.637532e+09,27.500114,-111.683222,...,NaN,NaN,NaN,NaN,NaN,NaN,0.037567,NaN,432515.553144,3.042015e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,JAS2,2021,11,21,22,59,55.615,1.637536e+09,27.500955,-111.683727,...,NaN,NaN,NaN,NaN,NaN,NaN,0.032731,NaN,432466.134565,3.042109e+06
3596,JAS2,2021,11,21,22,59,56.615,1.637536e+09,27.500955,-111.683727,...,NaN,NaN,NaN,NaN,NaN,NaN,0.032532,NaN,432466.134565,3.042109e+06
3597,JAS2,2021,11,21,22,59,57.615,1.637536e+09,27.500955,-111.683727,...,NaN,NaN,NaN,NaN,NaN,NaN,0.031519,NaN,432466.134565,3.042109e+06
3598,JAS2,2021,11,21,22,59,58.615,1.637536e+09,27.500955,-111.683727,...,NaN,NaN,NaN,NaN,NaN,NaN,0.034325,NaN,432466.134565,3.042109e+06


In [146]:
csv[['northing', 'easting', 'latitude(deg)', 'longitude(deg)']].max()

northing          4.325157e+05
easting           3.042109e+06
latitude(deg)     2.750095e+01
longitude(deg)   -1.116832e+02
dtype: float64

In [147]:
csv[['northing', 'easting', 'latitude(deg)', 'longitude(deg)']].min()

northing          4.324556e+05
easting           3.042015e+06
latitude(deg)     2.750011e+01
longitude(deg)   -1.116838e+02
dtype: float64

In [148]:
utm.from_latlon(2.750123e+01, -1.116835e+02)

(432488.74452372105, 3042139.010820424, 12, 'R')

In [149]:
utm.from_latlon(2.750109e+01, -1.116836e+02)

(432478.7815225468, 3042123.557202419, 12, 'R')

In [150]:
measured = csv[pd.notna(csv.oxygen_proj)]

In [151]:
-measured['depth(m)']

22     -1730.525
23     -1730.539
24     -1730.562
25     -1730.580
26     -1730.598
          ...   
3587   -1735.070
3588   -1735.070
3589   -1735.068
3590   -1735.066
3591   -1735.068
Name: depth(m), Length: 3570, dtype: float64

In [152]:
ring_mesh = utils.get_mesh('../data/ring_depth.csv', 1000)

In [163]:
csv[['latitude(deg)', 'longitude(deg)', 'depth(m)']]

,latitude(deg),longitude(deg),depth(m)
0,27.500114,-111.683222,1730.643
1,27.500114,-111.683222,1730.615
2,27.500114,-111.683222,1730.595
3,27.500114,-111.683222,1730.569
4,27.500114,-111.683222,1730.545
...,...,...,...
3595,27.500955,-111.683727,1735.068
3596,27.500955,-111.683727,1735.069
3597,27.500955,-111.683727,1735.069
3598,27.500955,-111.683727,1735.068


In [157]:
scatter = go.Scatter3d(
    x = csv.northing,
    y = csv.easting, 
    z = -csv['depth(m)'], 
    mode="markers", 
    marker=dict(color=csv['obs_proj'])
)

In [158]:
go.Figure(data=[ring_mesh, scatter])

In [50]:
csv_0 = csv.copy()

In [52]:
csv_0.index = csv_0["time(seconds since Jan 1 1970)"]

In [53]:
ct2.index = ct2.unix_time

In [55]:
combined = pd.merge_asof(csv_0, ct2, left_index=True, right_index=True)

In [57]:
combined['depth(m)']

time(seconds since Jan 1 1970)
1.637539e+09    1737.294
1.637539e+09    1737.294
1.637539e+09    1737.293
1.637539e+09    1737.294
1.637539e+09    1737.294
                  ...   
1.637543e+09    1722.508
1.637543e+09    1722.512
1.637543e+09    1722.521
1.637543e+09    1722.533
1.637543e+09    1722.550
Name: depth(m), Length: 3599, dtype: float64

In [60]:
fig = go.Figure(utils.get_mesh('../data/ring_depth.csv', 1000))

fig.show()